# Multi-objective Bayesian optimisation of ECRH/safety factor profiles

### Intro

We consider the selection of electron-cyclotron current profiles, where the goal is to obtain an optimal safety factor profile.

We use a constrained piecewise linear function with 12 parameters to represent the ECRH profile.

There are numerous competing objectives for the safety factor:

| Objective | Formulation |
|-----------|-------------|
| Minimise drop-off from on-axis value | $\exp(-\|q_0 - q_\text{min}\|)$ |
| Minimum close to 2.2, and not less than 2 | $\exp(-\|2.2 - q_\text{min}\|)$ if $q_\text{min} > 2$ else 0 |
| Minimum close to axis | $1 - \arg\min_\rho(q)$ |
| $q$ monotonically increasing | $\frac{\int_{q' > 0} q d\rho}{\int q d\rho}$ |
| $q'$ monotonically increasing | $\frac{\int_{q'' > 0} q' d\rho}{\int q' d\rho}$ |
| High shear at $q=3$ | $\arg_\rho(q = 3)$ |
| High shear at $q=4$ | $\arg_\rho(q = 4)$ |

Consequently, this is a multi-objective optimisation problem.

### Pareto optimality

In general, multi-objective optimisation problems do not have a single solution that simultaneously optimises each objective. Instead, there will be a 'Pareto set' of solutions:

> A solution is Pareto optimal if it is impossible to improve its performance in one objective without decreasing its performance in another.

Note that the Pareto set is technically infinite, and the probability that a solution is Pareto-optimal scales exponentially with the number of objectives. 
We aim to find a subset of Pareto-optimal solutions, which we can use to inform decision-making.

<center>
<img src="pareto_front.png" width=500>


### Multi-objective Bayesian optimisation

To find Pareto-optimal solutions, we use multi-objective Bayesian optimisation.

This method has two defining features:

1. **Surrogate model:** Gaussian process

    Each new trial point is used to update a probabilistic model that allows us to predict the performance of an unseen point.


2. **Acquisition function:** batch noisy expected hypervolume improvement

    For the next point, choose the point that has the highest expected increase in performance across all objectives.


### Results

Load data from BayesOpt run:

In [9]:
import numpy as np
from jetto_mobo import ecrh, genetic_algorithm
import plotly.graph_objects as go
import h5py

n_steps = 5
batch_size = 5
n_objectives = 7
n_parameters = 12
n_radial_points = 150

parameters = np.zeros((n_steps * batch_size, n_parameters))
objective_values = np.zeros((n_steps * batch_size, n_objectives))
converged_ecrh = np.zeros((n_steps * batch_size, 150))
converged_q = np.zeros((n_steps * batch_size, 150))

with h5py.File("../data/mobo/ga_piecewise_linear.hdf5") as hdf5:
    for i in np.arange(n_steps):
        for j in np.arange(batch_size):
            if not np.any(np.isnan(hdf5[f"bayesopt/{i+1}/value"][j])):
                objective_values[i*batch_size + j] = hdf5[f"bayesopt/{i+1}/value"][j]
                parameters[i*batch_size + j] = hdf5[f"bayesopt/{i+1}/ecrh_parameters"][j]
                converged_ecrh[i*batch_size + j] = hdf5[f"bayesopt/{i+1}/converged_ecrh"][j]
                converged_q[i*batch_size + j] = hdf5[f"bayesopt/{i+1}/converged_q"][j]
print(f"Loaded results from {n_steps} optimisation steps, each with {batch_size} points (totalling {n_steps * batch_size} evaluations).")
print(f"Input space: {n_parameters} dimensions")
print(f"Output (objective) space: {n_objectives} dimensions")

Loaded results from 5 optimisation steps, each with 5 points (totalling 25 evaluations).
Input space: 12 dimensions
Output (objective) space: 7 dimensions


Compute Pareto optimal points, discarding infeasible ones:

In [10]:
from jetto_mobo.utils import get_pareto_dominant_mask

pareto_dominant_mask = get_pareto_dominant_mask(objective_values, allow_zero=False)
n_pareto_optimal_points = np.sum(pareto_dominant_mask)

print(f"Of {n_steps * batch_size} evaluated solutions, {n_pareto_optimal_points} are Pareto optimal.")

Of 25 evaluated solutions, 8 are Pareto optimal.


Plot results:

In [26]:
from typing import Iterable
import numpy as np
from plotly import graph_objects as go 
from plotly.subplots import make_subplots
import plotly

figure = make_subplots(rows=2, cols=2, specs=[[{'rowspan': 2, 'type': 'polar'}, {}], [{}, {}]])

i = 1
for objective_value, ecrh, q in zip(objective_values[pareto_dominant_mask],
                                    converged_ecrh[pareto_dominant_mask],
                                    converged_q[pareto_dominant_mask]):
    figure.add_traces(
        [
            go.Scatterpolar(
                r=objective_value,
                theta=["Proximity of<br>q(0) to min(q)",
                        "Proximity of<br>min(q) to 2.2",
                        "Proximity of<br>argmin(q) to axis",
                        "Monotonic area<br>fraction of q",
                        "Monotonic area<br>fraction of dq",
                        "Rho of q=3",
                        "Rho of q=4"],
                legendgroup=i,
                name=f"Solution {i}",
                showlegend=False,
                fill="toself",
                line_color=plotly.colors.qualitative.T10[i-1]
            ),
            go.Scatter(
                x=np.linspace(0, 1, n_radial_points),
                y=ecrh,
                legendgroup=i,
                name=f"Solution {i}",
                showlegend=False,
                mode='lines',
                line_color=plotly.colors.qualitative.T10[i-1],
            ),
            go.Scatter(
                x=np.linspace(0, 1, n_radial_points),
                y=q,
                legendgroup=i,
                name=f"Solution {i}",
                showlegend=True,
                mode='lines',
                line_color=plotly.colors.qualitative.T10[i-1],
            ),
            go.Scatter(
                x=[np.linspace(0, 1, n_radial_points)[np.argmin(q)]],
                y=[np.min(q)],
                legendgroup=i,
                name=f"Solution {i} - minimum",
                mode='markers',
                line_color=plotly.colors.qualitative.T10[i-1],
                showlegend=False,
            )
        ],
        rows=[1, 1, 2, 2],
        cols=[1, 2, 2, 2]
    )
    i+=1

figure.update_layout(polar_radialaxis_range=[0, 1],
                     title="Pareto-optimal solutions to the ECRH / safety factor profile optimisation problem",
                     title_x=0.5)
figure.update_yaxes(title="ECRH power density", row=1, col=2)
figure.update_yaxes(range=[0, 12], title="Safety factor", row=2, col=2)
figure.update_xaxes(title="Normalised radius")
figure.show()

In [27]:
import os
if not os.path.exists("html"):
    os.makedirs("html")
figure.write_html("html/mobo.html")